In [1]:
import pandas as pd
from indicators.syncind import SyncInd
from indicators.classic import SMA, Alligator, SmoothMA, RSI, MACD, ATR, WMA, EMA, RMA, VolumeROC, KAMA, BollingerBands
from indicators import time_to_epoch, epoch_to_time
import datetime, time
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


/Users/anodic_passion/project_vault/108decillion_ext/.venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
import requests

def get_market_rates(symbol:str, timeframe:str, from_candle:int=0, previous_candle_count:int=100, eohlcv:bool=True, base_url:str="http://10.168.85.228:7767/data") -> np.ndarray:
    if timeframe not in ["1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"]:
        raise Exception('''Timeframe should be from -> "1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"''')

    if timeframe == "1m":
        _tf = 1
    elif timeframe == "5m":
        _tf = 5
    elif timeframe == "15m":
        _tf = 15
    elif timeframe == "30m":
        _tf = 30
    elif timeframe == "1h":
        _tf = 16385
    elif timeframe == "1d":
        _tf = 16408
    elif timeframe == "4h":
        _tf = 16388
    elif timeframe == "1w":
        _tf = 32769
    elif timeframe == "1mo":
        _tf = 49153
    else:
        raise Exception('''Timeframe should be from -> "1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"''')
    payload = {
        "symbol": symbol,
        "timeframe": _tf,
        "from_candle": from_candle,
        "previous": previous_candle_count
    }

    response = requests.post(base_url, json=payload)

    if response.status_code == 200:
        _data = response.json()['rates']
        # print("Bars received:", len(_data))
        if eohlcv:
            return np.array(_data)[:, :6]
        else:
            return np.array(_data)
    else:
        print("Error:", response.text)
        raise RuntimeError(response.text)

def get_market_range_rates(symbol:str, timeframe:str, from_candle:str="31-12-2025 00:00", previous_candle_count:int=100, eohlcv:bool=True, base_url:str="http://10.168.85.228:7767/data-range") -> np.ndarray:
    if timeframe not in ["1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"]:
        raise Exception('''Timeframe should be from -> "1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"''')

    if timeframe == "1m":
        _tf = 1
    elif timeframe == "5m":
        _tf = 5
    elif timeframe == "15m":
        _tf = 15
    elif timeframe == "30m":
        _tf = 30
    elif timeframe == "1h":
        _tf = 16385
    elif timeframe == "1d":
        _tf = 16408
    elif timeframe == "4h":
        _tf = 16388
    elif timeframe == "1w":
        _tf = 32769
    elif timeframe == "1mo":
        _tf = 49153
    else:
        raise Exception('''Timeframe should be from -> "1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"''')
    payload = {
        "symbol": symbol,
        "timeframe": _tf,
        "from-time": from_candle,
        "previous": previous_candle_count
    }

    response = requests.post(base_url, json=payload)

    if response.status_code == 200:
        _data = response.json()['rates']
        # print("Bars received:", len(_data))
        if eohlcv:
            return np.array(_data)[:, :6]
        else:
            return np.array(_data)
    else:
        print("Error:", response.text)
        raise RuntimeError(response.text)

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

PROMPT = "Hello DeepSeek, write a haiku about AI."

# Setup Chrome
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

driver.get("https://chat.deepseek.com")

wait = WebDriverWait(driver, 30)

# Wait for the input textarea (selector may change)
input_box = wait.until(
    EC.presence_of_element_located((By.TAG_NAME, "textarea"))
)

# Type prompt and press Enter
input_box.send_keys(PROMPT)
input_box.send_keys(Keys.ENTER)

# Optional: keep browser open
time.sleep(10)


In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

PROMPT = "Write a short imaginary story about artificial intelligence."

# Setup Chrome
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

wait = WebDriverWait(driver, 40)

# Open DeepSeek Chat
driver.get("https://chat.deepseek.com")

# Wait for input box
input_box = wait.until(
    EC.presence_of_element_located((By.TAG_NAME, "textarea"))
)

# Send prompt
input_box.send_keys(PROMPT)
input_box.send_keys(Keys.ENTER)

# ---- WAIT FOR RESPONSE ----
# This waits until at least one assistant message appears
wait.until(
    EC.presence_of_element_located(
        (By.CSS_SELECTOR, "div[class*='message'], div[class*='assistant']")
    )
)

# Give extra time for streaming to finish
time.sleep(5)

# ---- EXTRACT RESPONSE ----
# Grab all visible messages
messages = driver.find_elements(
    By.CSS_SELECTOR,
    "div[class*='_0f72b0b'], div[class*='assistant']"
)

# Print last assistant response
if messages:
    last_message = messages[-1].text
    print("\n===== DeepSeek Response =====\n")
    print(last_message)
else:
    print("No response found.")

# Keep browser open briefly
time.sleep(5)
driver.quit()



ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))